# Download White Wine Data From Vivino.com

Before running this download the file chromedriver, you will need to enter the path to this file below:

driver = webdriver.Chrome(executable_path='path', chrome_options=options)

if you are using Windows input the path to chromedriver under the if statement:

if os.name == 'nt'

otherwise enter it under the else statement

Chrome 59 or greater is required for the headless option to work.  Script should still run, just not headless.

Libraries:
* conda install -c conda-forge selenium
* https://sites.google.com/a/chromium.org/chromedriver/downloads

Note:  Red wine script included below, but is not completed.  I'm tired and want to go to bed.  Future Chris can deal with it.

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import os
import csv
import pandas as pd

options = webdriver.ChromeOptions(); options.add_argument('headless')
if os.name == 'nt':
    driver = webdriver.Chrome(executable_path='D:\orson\Downloads\chromedriver_win32\chromedriver', chrome_options=options)    
else:
    driver = webdriver.Chrome(executable_path='/home/chris/Downloads/chromedriver', chrome_options=options)

country_list = []; rating_list = []; popular_list = []; price_list = []
wine_list    = []; winery_list = []
site_list    = ['https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1NFDLTaywNTRWS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAb34PXA==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1NFbLTaywNTRTS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAcE0PYg==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1NFPLTaywNTJQS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAcCUPYA==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1MlDLTaywNTJVS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAcAMPYA==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1MlXLTaywNTVQS660LS1WKwCKF9smJ6qVJduWFJWmqpWXRMfaGgEAcIYPYw==',\
                'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1NVDLTawAUgZqyZW2pcVqBUCJYtvkRLWyZNuSotJUtfKS6FhbIwB7dQ-R']

driver.get(site_list[0])
df_name_list = ['wine','winery','region','country','rating','pricing','popularity']
df = pd.DataFrame(columns = df_name_list); number = 0

for site in range(len(site_list)):
    print("Starting Scrape of {0} of {1} Websites".format(site+1, len(site_list)))
    driver.get(site_list[site])

    lastHeight = driver.execute_script("return document.body.scrollHeight")
    total_time = 0; total_page_loads = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight and total_time > 15:
            print('  No new data on this site'); break
        time.sleep(0.25); total_time += 0.25
        if lastHeight < newHeight:
            if total_page_loads % 5 == 0:
                print("  Pages Loaded:   ", total_page_loads)
            total_page_loads += 1; total_time = 0; lastHeight = newHeight

    time.sleep(5.0)
    for i in range(1000):
        try:
            header          = '//section[@class="wine-explorer__results"]/div[{}]'.format(i+1)
            wine_path       = '/div[1]/div[1]/a[1]'   + '/span[1]'
            winery_path     = '/div[1]/div[1]/a[1]'   + '/span[2]'
            region_path     = '/div[1]/div[1]/div[1]' + '/a[3]'
            country_path    = '/div[1]/div[1]/div[1]' + '/a[2]'
            popularity_path = '/div[1]/div[1]/div[2]' + '/div[1]/div[2]/div[2]/div[2]/span[1]/span[1]'
            rating_path     = '/div[1]/div[1]/div[2]' + '/div[1]/div[2]/div[1]'
            pricing_path    = '/div[1]/div[1]/div[2]' + '/div[2]/div[2]/div[1]/span[1]'            

            wine       = driver.find_element_by_xpath(header + wine_path).text
            winery     = driver.find_element_by_xpath(header + winery_path).text
            popularity = driver.find_element_by_xpath(header + popularity_path).text
            rating     = driver.find_element_by_xpath(header + rating_path).text
            country    = driver.find_element_by_xpath(header + country_path).text
            region     = driver.find_element_by_xpath(header + region_path).text
            pricing    = driver.find_element_by_xpath(header + pricing_path).text
            entry      = [wine, winery, region, country, rating, pricing, popularity]
            df         = df.append(pd.DataFrame([entry],columns=df_name_list), ignore_index=True)
        except NoSuchElementException:
            pass
    
    print('Number of entries in the database: ', len(df))
    number += int(driver.find_element_by_xpath('//div[@class="query-summary__primary"]/span[3]').text)
    print("Total number of wines loaded: ", number, '\n\n')

df.to_csv('vivino_white_wine.csv')
print("\n\nThat's all Folks!")


Starting Scrape of 1 of 6 Websites
  Pages Loaded:    0
  Pages Loaded:    5
  Pages Loaded:    10
  Pages Loaded:    15
  Pages Loaded:    20
  No new data on this site
Number of entries in the database:  484
Total number of wines loaded:  485 


Starting Scrape of 2 of 6 Websites
  Pages Loaded:    0
  Pages Loaded:    5
  Pages Loaded:    10
  Pages Loaded:    15
  Pages Loaded:    20
  Pages Loaded:    25
  No new data on this site
Number of entries in the database:  1041
Total number of wines loaded:  1043 


Starting Scrape of 3 of 6 Websites
  Pages Loaded:    0
  Pages Loaded:    5
  Pages Loaded:    10
  Pages Loaded:    15
  Pages Loaded:    20
  Pages Loaded:    25
  No new data on this site
Number of entries in the database:  1698
Total number of wines loaded:  1700 


Starting Scrape of 4 of 6 Websites
  Pages Loaded:    0
  Pages Loaded:    5
  Pages Loaded:    10
  Pages Loaded:    15
  No new data on this site
Number of entries in the database:  2166
Total number of win

In [ ]:
# move = webdriver.ActionChains(driver)
# move = ActionChains(driver)
# slider = driver.find_element_by_xpath('//span[@class="ui-slider-handle ui-corner-all ui-state-default"]')

# elements = [driver.find_element_by_xpath('//span[@class="ui-slider-handle ui-corner-all ui-state-default"]'),
#             driver.find_element_by_xpath('//div[@class="query-summary__primary"]'),
#             driver.find_element_by_xpath('//span[@class="wine-explorer__filters__item__header__title"]')]
# for item in elements:
#     time.sleep(5)
#     print("Test")
#     move.move_to_element(item).perform()
# move.click_and_hold(slider).move_by_offset(20, 0).release()

# time.sleep(5)
# move.click_and_hold(slider

# time.sleep(5)
# move.click_and_hold(slider).move_by_offset(20, 0).release()




    
    
    
#     header_elements     = driver.find_element_by_class_name('wine-card__content')
#     price_elements      = driver.find_elements_by_xpath('//div[@class="wine-card__statistics"]/div[2]/div[2]/div[1]/span[1]')
#     country_elements    = driver.find_elements_by_xpath('//div[@class="location"]/a[2]')
#     popularity_elements = driver.find_elements_by_xpath('//div[@class="statistics-item__content__description"]/div[2]/span[1]/span[1]')
#     rating_elements     = driver.find_elements_by_xpath('//div[@class="wine-card__statistics"]/div[1]/div[2]/div[1]')
#     wine_elements       = driver.find_elements_by_xpath('//a[@class="wine-card__header"]/span[2]')
#     winery_elements     = driver.find_elements_by_xpath('//a[@class="wine-card__header"]/span[1]')
    
#     for card in header_elements:
#         print('\n\n', card)
#         print('Winery     : ', card.find_elements_by_xpath('//div[@class="wine-card__content"]/a[1]/span[1]').text)
#         print('Wine       : ', card.find_elements_by_xpath('//div[@class="wine-card__content"]/a[1]/span[2]').text)
#         print('Country    : ', card.find_elements_by_xpath('//div[1]/a[2]').text)
#         print('Region     : ', card.find_elements_by_xpath('//div[1]/a[3]').text)
#         print('Rating     : ', card.find_elements_by_xpath('//div[2]/div[1]').text)
#         print('Popularity : ', card.find_elements_by_xpath('//div[2]/div[1]/div[2]/div[2]/div[2]/span[1]/span[1]').text)
#         print('Pricy      : ', card.find_elements_by_xpath('//div[2]/div[2]/div[2]/div[1]/span[1]').text)
                
#     print('Testing    : ', driver.find_element_by_xpath('//div[@class="wine-card__content"][1]'))
#     print('Testing    : ', driver.find_element_by_xpath('//div[@class="wine-card__content"][2]'))
#     print(header_elements[1])
#     print(header_elements[0])


#     prices       = [x.text for x in price_elements]
#     countries    = [x.text for x in country_elements]
#     popularities = [x.text for x in popularity_elements]
#     ratings      = [x.text for x in rating_elements]
#     wineries     = [x.text for x in winery_elements]
#     wines        = [x.text for x in wine_elements]
#     number       = driver.find_element_by_xpath('//div[@class="query-summary__primary"]/span[3]').text

#     print("Sanity check should find ", number, " wines in each list")
#     print(len(countries)); 
#     while len(countries) < int(number):
#         countries.append('')
#     country_list = country_list + countries
#     print(len(ratings));    
#     while len(ratings) < int(number):
#         ratings.append('')
#     rating_list  = rating_list  + ratings
#     print(len(prices));     
#     while len(prices) < int(number):
#         prices.append('')
#     price_list   = price_list   + prices
#     print(len(popularities)); 
#     while len(popularities) < int(number):
#         popularities.append('')
#     popular_list = popular_list + popularities
#     print(len(wines))
#     while len(wines) < int(number):
#         wines.append('')
#     wine_list = wine_list + wines
#     print(len(wineries))
#     while len(wineries) < int(number):
#         wineries.append('')
#     winery_list = winery_list + wineries


# outputter = pd.DataFrame({'wine':wine_list, 'winery':winery_list, 'country':country_list, 'rating':rating_list, \
#                           'price':price_list, 'popularity':popular_list})
# outputter.to_csv('vivino_white_wine.csv')